In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [37]:
df_hedged_returns = pd.read_csv("data/q3/hedged_returns.csv",
                                   parse_dates=["date"],
                                   index_col="date")

df = df_hedged_returns.rename(columns={
    "australia_hedged_return_usd":   "AUS",
    "france_hedged_return_usd":      "FRA",
    "germany_hedged_return_usd":     "GER",
    "japan_hedged_return_usd":       "JPN",
    "switzerland_hedged_return_usd": "SWI",
    "uk_hedged_return_usd":          "UK"
})


# Convert the 'date' column to datetime format and shift to the first day of the next month
df_tbills = pd.read_csv('data/monthly_returns/tbills.csv')
df_tbills['date'] = pd.to_datetime(df_tbills['date']) + pd.offsets.MonthBegin(1)
df_tbills.set_index('date', inplace=True)
df_tbills.to_csv('data/preprocess/tbills.csv')

# To match the hedged return we drop the last two rows (January 2025 and December 2024)
tbill = df_tbills.rename(columns={"rf": "TBill1Mo"})["TBill1Mo"].iloc[:-2]

#
#   div_df = pd.read_csv("data/q3/div_returns.csv", parse_dates=["date"], index_col="date")
#   div = div_df["DIV_return"]   # a pandas Series of DIV returns, index matches 2002-05-01 ... etc.
#

one_plus = 1.0 + df

### 4 Equity Index Momentum Strategy (MOM)

In [41]:
# Compute 11-month lookback cumulative return, then shift by 1 to impose 1-month lag:
cumprod_11 = one_plus.rolling(window=12).apply(np.prod, raw=True).shift(1) - 1

# Rank each row (month) ascending
ranks_11 = cumprod_11.rank(axis=1, method="first", ascending=True)

# Compute scaling factor Z so that ∑_{i: w>0} w = +1 and ∑_{i: w<0} w = -1.
N = df.shape[1]             
mean_rank = (N + 1) / 2.0    
if N % 2 == 0:
    Z = 8.0 / (N**2)       
else:
    Z = 8.0 / (N**2 - 1)   

# Compute MOM weights
w_mom = Z * (ranks_11 - mean_rank)

# Strategy return at month 
mom_ret = (w_mom * df).sum(axis=1, skipna=False)

# Decompose the “long leg” (w>0) and “short leg” (w<0) returns:
long_w_mom  = w_mom.clip(lower=0)
short_w_mom = w_mom.clip(upper=0)
mom_long_ret  = (long_w_mom  * df).sum(axis=1, skipna=False)
mom_short_ret = (short_w_mom * df).sum(axis=1, skipna=False)

# Align with T-Bill series, drop NaNs for clean statistics:
mom_df = pd.DataFrame({
    "MOM":   mom_ret,
    "LONG":  mom_long_ret,
    "SHORT": mom_short_ret,
    "TBill": tbill
}).dropna()

# Helper to compute annualized mean, vol, and Sharpe
def annualized_stats(returns, rf, period = 12):
    """
    Given two pandas Series of monthly returns (decimal):
      - returns: strategy returns
      - rf:      risk-free returns
    Returns (annualized_mean, annualized_vol, annualized_sharpe).
    """
    mean_annual_return = period * returns.mean()
    std_annual_return = np.sqrt(period) * returns.std()
    rf_annual = period * rf.mean()
    excess_ret = mean_annual_return - rf_annual
    sharpe_ratio = excess_ret / std_annual_return
    return mean_annual_return, std_annual_return, sharpe_ratio

mean_mom, sigma_mom, sharpe_mom = annualized_stats(mom_df["MOM"], mom_df["TBill"])
mean_mL,  sigma_mL,  sharpe_mL  = annualized_stats(mom_df["LONG"], mom_df["TBill"])
mean_mS,  sigma_mS,  sharpe_mS  = annualized_stats(mom_df["SHORT"],mom_df["TBill"])

# One-sample t-test: H0: mean(MOM) = 0
t_stat_mom, p_val_mom = stats.ttest_1samp(mom_df["MOM"], popmean=0.0)

# Regress MOM on DIV (Newey-West errors, lag=1)
#reg_mom_div = pd.DataFrame({
#    "MOM": mom_ret,
#    "DIV": div
#}).dropna()

#X_m = sm.add_constant(reg_mom_div["DIV"])
#y_m = reg_mom_div["MOM"]
#mom_on_div = sm.OLS(y_m, X_m).fit(cov_type="HAC", cov_kwds={"maxlags": 1})

print("=== MOMENTUM (MOM) Strategy Statistics ===")
print(f"Overall MOM       : Annualized Return = {mean_mom:.2%}, "
      f"Volatility = {sigma_mom:.2%}, Sharpe = {sharpe_mom:.2f}")
print(f"MOM Long Leg      : Annualized Return = {mean_mL:.2%}, "
      f"Volatility = {sigma_mL:.2%}, Sharpe = {sharpe_mL:.2f}")
print(f"MOM Short Leg     : Annualized Return = {mean_mS:.2%}, "
      f"Volatility = {sigma_mS:.2%}, Sharpe = {sharpe_mS:.2f}")
print(f"MOM mean-return t-stat = {t_stat_mom:.3f}, p-value = {p_val_mom:.3f}\n")

#print(">>> Regression of MOM on DIV:")
#print(mom_on_div.summary())
#print("\n")


=== MOMENTUM (MOM) Strategy Statistics ===
Overall MOM       : Annualized Return = 28.29%, Volatility = 15.10%, Sharpe = 1.78
MOM Long Leg      : Annualized Return = 28.89%, Volatility = 17.84%, Sharpe = 1.54
MOM Short Leg     : Annualized Return = -0.59%, Volatility = 15.85%, Sharpe = -0.13
MOM mean-return t-stat = 8.703, p-value = 0.000



In [ ]:
# a)

In [3]:
# b)

In [4]:
# c)

### 5 Equity Index Long Term Reversal strategy (REV)

In [39]:
# Compute rolling 60-month product and rolling 12-month product:
rolling48 = one_plus.rolling(window=60).apply(lambda x: np.prod(x[:-11]), raw=False).shift(1)  - 1

# Rank each month ascending 
ranks_48 = rolling48.rank(axis=1, method="first", ascending=True)

# Compute scaling factor Z so that ∑_{i: w>0} w = +1 and ∑_{i: w<0} w = -1.
N = df.shape[1]             
mean_rank = (N + 1) / 2.0    
if N % 2 == 0:
    Z = 8.0 / (N**2)       
else:
    Z = 8.0 / (N**2 - 1) 
    
# Compute MOM weights
w_rev = Z * (mean_rank - ranks_48)

# Strategy return at t: sum_i [ w_rev[i,t] * df[i,t] ]
rev_ret = (w_rev * df).sum(axis=1, skipna=False)

# Decompose “long leg” and “short leg” for REV:
long_w_rev  = w_rev.clip(lower=0)
short_w_rev = w_rev.clip(upper=0)
rev_long_ret  = (long_w_rev  * df).sum(axis=1, skipna=False)
rev_short_ret = (short_w_rev * df).sum(axis=1, skipna=False)

# Align with T-Bill, drop NaNs:
rev_df = pd.DataFrame({
    "REV":   rev_ret,
    "LONG":  rev_long_ret,
    "SHORT": rev_short_ret,
    "TBill": tbill
}).dropna()

mean_rev, sigma_rev, sharpe_rev = annualized_stats(rev_df["REV"], rev_df["TBill"])
mean_rL,  sigma_rL,  sharpe_rL  = annualized_stats(rev_df["LONG"], rev_df["TBill"])
mean_rS,  sigma_rS,  sharpe_rS  = annualized_stats(rev_df["SHORT"],rev_df["TBill"])

# One-sample t-test for REV mean
t_stat_rev, p_val_rev = stats.ttest_1samp(rev_df["REV"], popmean=0.0)

# Regress REV on DIV (Newey-West lag=1)
#reg_rev_div = pd.DataFrame({
#    "REV": rev_ret,
#    "DIV": div
#}).dropna()

#X_r = sm.add_constant(reg_rev_div["DIV"])
#y_r = reg_rev_div["REV"]
#rev_on_div = sm.OLS(y_r, X_r).fit(cov_type="HAC", cov_kwds={"maxlags": 1})

print("=== REVERSAL (REV) Strategy Statistics ===")
print(f"Overall REV       : Annualized Return = {mean_rev:.2%}, "
      f"Volatility = {sigma_rev:.2%}, Sharpe = {sharpe_rev:.2f}")
print(f"REV Long Leg      : Annualized Return = {mean_rL:.2%}, "
      f"Volatility = {sigma_rL:.2%}, Sharpe = {sharpe_rL:.2f}")
print(f"REV Short Leg     : Annualized Return = {mean_rS:.2%}, "
      f"Volatility = {sigma_rS:.2%}, Sharpe = {sharpe_rS:.2f}")
print(f"REV mean-return t-stat = {t_stat_rev:.3f}, p-value = {p_val_rev:.3f}\n")

#print(">>> Regression of REV on DIV:")
#print(rev_on_div.summary())

=== REVERSAL (REV) Strategy Statistics ===
Overall REV       : Annualized Return = -13.74%, Volatility = 15.04%, Sharpe = -0.99
REV Long Leg      : Annualized Return = 2.89%, Volatility = 15.26%, Sharpe = 0.11
REV Short Leg     : Annualized Return = -16.63%, Volatility = 18.04%, Sharpe = -0.99
REV mean-return t-stat = -3.831, p-value = 0.000



In [5]:
# a)

In [6]:
# b) 

In [7]:
# c)